In [ ]:
# ✅ 1. Install everything needed
!pip install faiss-cpu sentence-transformers transformers accelerate

# ✅ 2. Import libraries
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# ✅ 3. Load models (separately!)
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # For embeddings
tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-2')  # For generation
llm = AutoModelForCausalLM.from_pretrained('microsoft/phi-2', device_map="auto", torch_dtype=torch.float16)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

def embed_chunks(chunks):
    texts = [chunk["text"] for chunk in chunks]
    embeddings = model.encode(texts, show_progress_bar=True)

    # Append embedding to each chunk
    for i in range(len(chunks)):
        chunks[i]["embedding"] = embeddings[i].tolist()  # Convert to JSON-safe list

    return chunks


In [ ]:
# chunks = [
#     {"page_number": 1, "text": "RTCU is a rugged and programmable IoT controller designed for industrial-grade communication and integration."},
#     {"page_number": 2, "text": "It supports MQTT, HTTP, and RESTful APIs, making it suitable for SCADA and cloud integration."},
#     {"page_number": 3, "text": "The RTCU platform enables remote firmware updates and edge processing capabilities."}
# ]

chunks = embed_chunks
# Embed each chunk
vectors = embedder.encode([chunk["text"] for chunk in chunks], show_progress_bar=True)
for i, chunk in enumerate(chunks):
    chunk["embedding"] = vectors[i].tolist()


TypeError: 'function' object is not iterable

In [ ]:
def build_faiss_index(chunks):
    dim = len(chunks[0]["embedding"])
    index = faiss.IndexFlatL2(dim)
    metadata = [ {"page_number": c["page_number"]} for c in chunks ]
    texts = [c["text"] for c in chunks]
    index.add(np.array([np.array(c["embedding"], dtype=np.float32) for c in chunks]))
    return index, texts, metadata

index, texts, metadata = build_faiss_index(chunks)


In [ ]:
def search_faiss(query, embedder_model, index, texts, metadata, top_k=3):
    query_embedding = embedder_model.encode([query])[0].astype("float32")
    distances, indices = index.search(np.array([query_embedding]), top_k)

    results = []
    for idx in indices[0]:
        results.append({
            "text": texts[idx],
            "metadata": metadata[idx]
        })

    return results

def build_rag_prompt(query, context_chunks):
    context = "\n\n".join(chunk["text"] for chunk in context_chunks)
    prompt = f"""
You are a helpful assistant. Use the information below to answer the question.

### Context:
{context}

### Question:
{query}

### Answer:"""
    return prompt.strip()

def generate_answer(prompt, max_tokens=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)
    outputs = llm.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Answer:")[-1].strip()


In [ ]:
def rag_pipeline(query):
    results = search_faiss(query, embedder, index, texts, metadata, top_k=3)
    prompt = build_rag_prompt(query, results)
    answer = generate_answer(prompt)
    return answer, results


In [ ]:
# query = "What are the integration features of RTCU?"
query = input("Enter your query: ")
answer, retrieved_context = rag_pipeline(query)

print("🔍 Answer:\n")
print(answer)
